In [1]:
#参考　
# https://qiita.com/teru855/items/8346a94abde86a842a1b


# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))


# データを入力する　macでバックスラッシュはoption+¥
df= pd.read_csv("../input/train.tsv", delimiter="\t")
df.head()

# Any results you write to the current directory are saved as output.

sample_submission.csv
test.tsv
train.tsv



,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,NaN,10.0,1,No description yet
1,1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,0,This keyboard is in great condition and works ...
2,2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.0,1,Adorable top with a hint of lace and a key hol...
3,3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,NaN,35.0,1,New with tags. Leather horses. Retail for [rm]...
4,4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,NaN,44.0,0,Complete with certificate of authenticity


In [2]:
#  ブランドの種類
len(df.brand_name.unique())

4810

In [3]:
# 価格の種類
len(df.price.unique())

828

In [4]:
# コンディションの種類
df.item_condition_id.unique()

# カテゴリごとの、最高価格、最低価格を調べる
#カテゴリは大分類、中分類、小分類になっているはず
#アプローチは、分類を数値化して一番似ている文字の値付けを真似る感じだろうな
#カテゴリごとのテーブルをもてばいいのでは


array([3, 1, 2, 4, 5])

In [5]:
# アイテムの種類
len(df.train_id.unique())

# 100万件のデータがあるってことか、すごいな

1482535

In [6]:
# 最大2000ドルか、、、20万円ってことだな
df.price.max()

2009.0

In [7]:
# 最低価格は、無料ってこと
df.price.min()


0.0

In [8]:
s = df['price'].value_counts()
print(s)

# 価格分布をみて、もっとも当てはまりそうな価格とのマッチングをみればいいんじゃないかな、だいたいの価格がわかればいいんでしょ


10.0      99416
12.0      78986
14.0      76655
16.0      66413
9.0       63643
8.0       61564
15.0      58082
20.0      56323
7.0       52268
18.0      41243
24.0      40797
11.0      39786
19.0      37729
13.0      36800
26.0      33531
25.0      32450
6.0       32260
5.0       31502
21.0      28996
22.0      28012
30.0      25386
29.0      21793
17.0      21334
34.0      18816
36.0      18718
3.0       18703
31.0      17414
40.0      16545
4.0       16139
35.0      15943
          ...  
1299.0        1
671.0         1
719.0         1
1165.0        1
774.0         1
72.5          1
771.0         1
770.0         1
452.0         1
766.0         1
1058.0        1
764.0         1
760.0         1
1064.0        1
1070.0        1
755.0         1
754.0         1
1081.0        1
1095.0        1
749.0         1
745.0         1
68.5          1
744.0         1
816.0         1
1140.0        1
1150.0        1
31.5          1
45.5          1
727.0         1
378.0         1
Name: price, Length: 828

In [32]:
# 値が0の要素数
s = df['price'].value_counts()
#len(s)

s.head(60)



10.0    99416
12.0    78986
14.0    76655
16.0    66413
9.0     63643
8.0     61564
15.0    58082
20.0    56323
7.0     52268
18.0    41243
24.0    40797
11.0    39786
19.0    37729
13.0    36800
26.0    33531
25.0    32450
6.0     32260
5.0     31502
21.0    28996
22.0    28012
30.0    25386
29.0    21793
17.0    21334
34.0    18816
36.0    18718
3.0     18703
31.0    17414
40.0    16545
4.0     16139
35.0    15943
28.0    14916
23.0    13720
39.0    11364
46.0    11141
44.0    10210
27.0    10173
45.0    10167
32.0     9704
50.0     9517
41.0     9362
56.0     8382
49.0     6902
38.0     6838
51.0     6688
54.0     6509
60.0     6221
33.0     5807
66.0     5355
55.0     5328
42.0     4487
65.0     4148
59.0     3950
61.0     3766
64.0     3469
37.0     3295
70.0     3213
76.0     3184
75.0     3000
71.0     2953
48.0     2887
Name: price, dtype: int64

In [9]:
df.price.describe()

count    1.482535e+06
mean     2.673752e+01
std      3.858607e+01
min      0.000000e+00
25%      1.000000e+01
50%      1.700000e+01
75%      2.900000e+01
max      2.009000e+03
Name: price, dtype: float64

In [34]:
df["item_description"]

0                                         No description yet
1          This keyboard is in great condition and works ...
2          Adorable top with a hint of lace and a key hol...
3          New with tags. Leather horses. Retail for [rm]...
4                  Complete with certificate of authenticity
5          Banana republic bottoms, Candies skirt with ma...
6          Size small but straps slightly shortened to fi...
7          You get three pairs of Sophie cheer shorts siz...
8           Girls Size small Plus green. Three shorts total.
9          I realized his pants are on backwards after th...
10              0.25 oz Full size is 1oz for [rm] in Sephora
11         (5) new vs pink body mists (2.5 oz each) Fresh...
12                                       Xl, great condition
13                                        No description yet
14         Authentic. Suede fringe boots. Great condition...
15         Brand new. Deluxe travel size products. Contai...
16         2 glitter eye

In [40]:
# 価格ごとに並び替える
# 出現順に１０段階にわける
#　そのレベルの特徴量を出す　説明文のテキスト、内容、とか
#　そのレベルの特徴量って何かわかるものなのかな
#　単語の羅列を入れてみて、何かわかるのか？
#　組み合わせなのか、
#　使われている単語の、ヒストグラムをつくる？のか
#　なんの違いも生まれないんじゃないかな
#　道の単語によって、shipping のアイテムの重要度が変わってくる感じだな
# そんなアルゴリズムをどう書けばいいのだ？
# 巨大な単語超から、キーワードごとの値を出すところなんじゃないか

a= pd.read_csv("../input/sample_submission.csv")
a.head()
a.tail()


,test_id,price
693354,693354,26.738
693355,693355,26.738
693356,693356,26.738
693357,693357,26.738
693358,693358,26.738


In [41]:
#df.loc[df.name == 'socks', 'price'] = 500
a.loc[a.test_id == 1, 'price'] = 10
a.head()

,test_id,price
0,0,26.738
1,1,10.000
2,2,26.738
3,3,26.738
4,4,26.738


In [44]:
#df.loc[df.name == 'socks', 'price'] = 500
a.loc[a.test_id != -1, 'price'] = 10
a.head()

,test_id,price
0,0,10.0
1,1,10.0
2,2,10.0
3,3,10.0
4,4,10.0


In [51]:
#a.to_csv("mercari_1.csv")
a.to_csv("mercari_2.csv", index=False)